In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import arcpy
import urllib2
from pyproj import Proj, transform
import xmltodict
import mechanize

#Import and Standardize Data

In [2]:
#rootname = "/media/p/5F5B-8FCB/PROJECTS/UMAR/Phase_II/Data/chem/" #thumb on ubuntu
rootname = "E:\\PROJECTS\\UMAR\\Phase_II\\Data\\chem\\" #thumb on windows

In [3]:
WQPResultsFile = rootname + "result.csv"
WQPStationFile = rootname + "station.csv"
SDWISFile = rootname + "SDWIS_Cache.txt"
AGStationsFile = rootname + "AG_Stations_Cache.csv"
AGResultsFile = rootname + "AG_byparam.csv"
UGSFile = rootname + "UGS_Cache.txt"
STORLegStatFile = rootname + "UT_Cache_sta_001.txt"
STORLegResFile = rootname + "UT_Cache_res_001.txt"
STORParamFile = rootname + "parameter.txt"

In [4]:
fielddata = rootname + "FieldData.xlsx"
statelabresults0 = rootname + "utgs1402.txt"
statelabresults1 = rootname + "utgs1403.txt"
statelabresults2 = rootname + "utgs1501.txt"
statelabstations = rootname + "UtahStateLabStations.xlsx"

##WQP

http://waterqualitydata.us/Station/search?statecode=US%3A49&countycode=US%3A49%3A005&sampleMedia=Water&characteristicType=Information%3BInorganics%2C+Major%2C+Metals%3BInorganics%2C+Major%2C+Non-metals%3BInorganics%2C+Minor%2C+Metals%3BInorganics%2C+Minor%2C+Non-metals%3BNot+Assigned%3BNutrient%3BPhysical%3BStable+Isotopes&mimeType=csv&zip=yes&sorted=no


http://waterqualitydata.us/Result/search?statecode=US%3A49&countycode=US%3A49%3A005&sampleMedia=Water&characteristicType=Information%3BInorganics%2C+Major%2C+Metals%3BInorganics%2C+Major%2C+Non-metals%3BInorganics%2C+Minor%2C+Metals%3BInorganics%2C+Minor%2C+Non-metals%3BNot+Assigned%3BNutrient%3BPhysical%3BStable+Isotopes&mimeType=csv&zip=yes&sorted=no

In [5]:
Rdtypes = {"OrganizationIdentifier":np.str_, "OrganizationFormalName":np.str_, "ActivityIdentifier":np.str_, 
           "ActivityStartTime/Time":np.str_,
           "ActivityTypeCode":np.str_, "ActivityMediaName":np.str_, "ActivityMediaSubdivisionName":np.str_, 
           "ActivityStartDate":np.str_, "ActivityStartTime/Time":np.str_, "ActivityStartTime/TimeZoneCode":np.str_, 
           "ActivityEndDate":np.str_, "ActivityEndTime/Time":np.str_, "ActivityEndTime/TimeZoneCode":np.str_, 
           "ActivityDepthHeightMeasure/MeasureValue":np.float16, "ActivityDepthHeightMeasure/MeasureUnitCode":np.str_, 
           "ActivityDepthAltitudeReferencePointText":np.str_, "ActivityTopDepthHeightMeasure/MeasureValue":np.float16, 
           "ActivityTopDepthHeightMeasure/MeasureUnitCode":np.str_, 
           "ActivityBottomDepthHeightMeasure/MeasureValue":np.float16, 
           "ActivityBottomDepthHeightMeasure/MeasureUnitCode":np.str_, 
           "ProjectIdentifier":np.str_, "ActivityConductingOrganizationText":np.str_, 
           "MonitoringLocationIdentifier":np.str_, "ActivityCommentText":np.str_, 
           "SampleAquifer":np.str_, "HydrologicCondition":np.str_, "HydrologicEvent":np.str_, 
           "SampleCollectionMethod/MethodIdentifier":np.str_, "SampleCollectionMethod/MethodIdentifierContext":np.str_, 
           "SampleCollectionMethod/MethodName":np.str_, "SampleCollectionEquipmentName":np.str_, 
           "ResultDetectionConditionText":np.str_, "CharacteristicName":np.str_, "ResultSampleFractionText":np.str_, 
           "ResultMeasureValue":np.str_, "ResultMeasure/MeasureUnitCode":np.str_, "MeasureQualifierCode":np.str_, 
           "ResultStatusIdentifier":np.str_, "StatisticalBaseCode":np.str_, "ResultValueTypeName":np.str_, 
           "ResultWeightBasisText":np.str_, "ResultTimeBasisText":np.str_, "ResultTemperatureBasisText":np.str_, 
           "ResultParticleSizeBasisText":np.str_, "PrecisionValue":np.str_, "ResultCommentText":np.str_, 
           "USGSPCode":np.str_, "ResultDepthHeightMeasure/MeasureValue":np.float16, 
           "ResultDepthHeightMeasure/MeasureUnitCode":np.str_, "ResultDepthAltitudeReferencePointText":np.str_, 
           "SubjectTaxonomicName":np.str_, "SampleTissueAnatomyName":np.str_, 
           "ResultAnalyticalMethod/MethodIdentifier":np.str_, "ResultAnalyticalMethod/MethodIdentifierContext":np.str_, 
           "ResultAnalyticalMethod/MethodName":np.str_, "MethodDescriptionText":np.str_, "LaboratoryName":np.str_, 
           "AnalysisStartDate":np.str_, "ResultLaboratoryCommentText":np.str_, 
           "DetectionQuantitationLimitTypeName":np.str_, "DetectionQuantitationLimitMeasure/MeasureValue":np.str_, 
           "DetectionQuantitationLimitMeasure/MeasureUnitCode":np.str_, "PreparationStartDate":np.str_, 
           "ProviderName":np.str_} 

dt = [6,56,61]

In [6]:
WQP = pd.read_csv(WQPResultsFile, dtype=Rdtypes, parse_dates=dt)

In [7]:
ResFieldDict = {"AnalysisStartDate":"AnalysisDate", "ResultAnalyticalMethod/MethodIdentifier":"AnalytMeth", 
                "ResultAnalyticalMethod/MethodName":"AnalytMethId", "ResultDetectionConditionText":"DetectCond", 
                "ResultLaboratoryCommentText":"LabComments", "LaboratoryName":"LabName", 
                "DetectionQuantitationLimitTypeName":"LimitType", "DetectionQuantitationLimitMeasure/MeasureValue":"MDL", 
                "DetectionQuantitationLimitMeasure/MeasureUnitCode":"MDLUnit", "MethodDescriptionText":"MethodDescript", 
                "OrganizationIdentifier":"OrgId", "OrganizationFormalName":"OrgName", "CharacteristicName":"Param", 
                "ProjectIdentifier":"ProjectId", "MeasureQualifierCode":"QualCode", "ResultCommentText":"ResultComment", 
                "ResultStatusIdentifier":"ResultStatus", "ResultMeasureValue":"ResultValue", 
                "ActivityCommentText":"SampComment", "ActivityDepthHeightMeasure/MeasureValue":"SampDepth", 
                "ActivityDepthAltitudeReferencePointText":"SampDepthRef", 
                "ActivityDepthHeightMeasure/MeasureUnitCode":"SampDepthU", "SampleCollectionEquipmentName":"SampEquip", 
                "ResultSampleFractionText":"SampFrac", "ActivityStartDate":"SampleDate", "ActivityIdentifier":"SampleId", 
                "ActivityStartTime/Time":"SampleTime", "ActivityMediaSubdivisionName":"SampMedia", 
                "SampleCollectionMethod/MethodIdentifier":"SampMeth", "SampleCollectionMethod/MethodName":"SampMethName", 
                "ActivityTypeCode":"SampType", "MonitoringLocationIdentifier":"StationId", 
                "ResultMeasure/MeasureUnitCode":"Unit", "USGSPCode":"USGSPCode",
                "ActivityStartDate":"StartDate","ActivityStartTime/Time":"StartTime"} 

In [8]:
WQP.rename(columns=ResFieldDict,inplace=True)

In [9]:
resdroplist = ["ActivityBottomDepthHeightMeasure/MeasureUnitCode", "ActivityBottomDepthHeightMeasure/MeasureValue", 
               "ActivityConductingOrganizationText", "ActivityEndDate", "ActivityEndTime/Time", 
               "ActivityEndTime/TimeZoneCode", "ActivityMediaName", "ActivityStartTime/TimeZoneCode", 
               "ActivityTopDepthHeightMeasure/MeasureUnitCode", "ActivityTopDepthHeightMeasure/MeasureValue", 
               "HydrologicCondition", "HydrologicEvent", "PrecisionValue", "PreparationStartDate", "ProviderName", 
               "ResultAnalyticalMethod/MethodIdentifierContext", "ResultDepthAltitudeReferencePointText", 
               "ResultDepthHeightMeasure/MeasureUnitCode", "ResultDepthHeightMeasure/MeasureValue", 
               "ResultParticleSizeBasisText", "ResultTemperatureBasisText", 
               "ResultTimeBasisText", "ResultValueTypeName", "ResultWeightBasisText", "SampleAquifer", 
               "SampleCollectionMethod/MethodIdentifierContext", "SampleTissueAnatomyName", "StatisticalBaseCode", 
               "SubjectTaxonomicName","StartTime","StartDate","StartTime","StartDate"] 

In [10]:
def datetimefix(x,format):
    d = str(x[0]).lstrip().rstrip()[0:10]
    t = str(x[1]).lstrip().rstrip()[0:5].zfill(5)
    try:
        int(d[0:2])
    except(ValueError,TypeError,NameError):
        return np.nan
    try:
        int(t[0:2])
        int(t[3:5])
    except(ValueError,TypeError,NameError):
        t = "00:00"
   
    if int(t[0:2])>23:
        t = "00:00"
    elif int(t[3:5])>59:
        t = "00:00"
    else:
        t = t[0:2].zfill(2) + ":" + t[3:5]
    return datetime.datetime.strptime(d + " " + t, format)
    

In [11]:
WQP["SampleDate"] = WQP[["StartDate","StartTime"]].apply(lambda x: datetimefix(x,"%Y-%m-%d %H:%M"),1)

In [12]:
WQP.drop(resdroplist,inplace=True,axis=1)

In [13]:
WQP['ResultValue'] = WQP['ResultValue'].convert_objects(convert_numeric=True)
WQP['MDL'] = WQP['MDL'].convert_objects(convert_numeric=True)

In [14]:
WQP['StationId'] = WQP['StationId'].str.replace('_WQX-','-')

In [15]:
def unitfix(x):
    z = str(x).lower()
    if z == "ug/l":
        return "mg/l"
    elif z == "mg/l":
        return "mg/l"
    else:
        return x

WQP.Unit = WQP.Unit.apply(lambda x: str(x).rstrip(), 1)
WQP.ResultValue = WQP[["ResultValue","Unit"]].apply(lambda x: x[0]/1000 if str(x[1]).lower()=="ug/l" else x[0], 1)
WQP.Unit = WQP.Unit.apply(lambda x: unitfix(x),1)

In [16]:
def parnorm(x):
    p = str(x[0]).rstrip().lstrip().lower()
    u = str(x[2]).rstrip().lstrip().lower()
    if p == 'nitrate' and u == 'mg/l as n':
        return 'Nitrate', x[1]*4.427, 'mg/l'
    elif p == 'nitrite' and u == 'mg/l as n':
        return 'Nitrite', x[1]*3.285, 'mg/l'
    elif p == 'ammonia-nitrogen' or p == 'ammonia-nitrogen as n' or p == 'ammonia and ammonium':
        return 'Ammonium', x[1]*1.288, 'mg/l'
    elif p == 'ammonium' and u == 'mg/l as n':
        return 'Ammonium', x[1]*1.288, 'mg/l'
    elif p == 'sulfate as s':
        return 'Sulfate', x[1]*2.996, 'mg/l'
    elif p in ('phosphate-phosphorus', 'phosphate-phosphorus as p','orthophosphate as p'):
        return 'Phosphate', x[1]*3.066, 'mg/l'
    elif (p == 'phosphate' or p == 'orthophosphate') and u == 'mg/l as p':
        return 'Phosphate', x[1]*3.066, 'mg/l'
    elif u == 'ug/l':
        return x[0], x[1]/1000, 'mg/l'
    else:
        return x[0], x[1], str(x[2]).rstrip()

WQP['Param'], WQP['ResultValue'], WQP['Unit'] = zip(*WQP[['Param','ResultValue','Unit']].apply(lambda x: parnorm(x),1))

In [17]:
WQPStat = pd.read_csv(WQPStationFile)

In [18]:
StatFieldDict = {"MonitoringLocationIdentifier":"StationId", "AquiferName":"Aquifer", "AquiferTypeName":"AquiferType", 
             "ConstructionDateText":"ConstDate", "CountyCode":"CountyCode", "WellDepthMeasure/MeasureValue":"Depth", 
             "WellDepthMeasure/MeasureUnitCode":"DepthUnit", "VerticalMeasure/MeasureValue":"Elev", 
             "VerticalAccuracyMeasure/MeasureValue":"ElevAcc", "VerticalAccuracyMeasure/MeasureUnitCode":"ElevAccUnit", 
             "VerticalCollectionMethodName":"ElevMeth", "VerticalCoordinateReferenceSystemDatumName":"ElevRef", 
             "VerticalMeasure/MeasureUnitCode":"ElevUnit", "FormationTypeText":"FmType", 
             "WellHoleDepthMeasure/MeasureValue":"HoleDepth", "WellHoleDepthMeasure/MeasureUnitCode":"HoleDUnit", 
             "HorizontalAccuracyMeasure/MeasureValue":"HorAcc", "HorizontalAccuracyMeasure/MeasureUnitCode":"HorAccUnit", 
             "HorizontalCollectionMethodName":"HorCollMeth", "HorizontalCoordinateReferenceSystemDatumName":"HorRef", 
             "HUCEightDigitCode":"HUC8", "LatitudeMeasure":"Lat_Y", "LongitudeMeasure":"Lon_X", 
             "OrganizationIdentifier":"OrgId", "OrganizationFormalName":"OrgName", "StateCode":"StateCode", 
             "MonitoringLocationDescriptionText":"StationComment", "MonitoringLocationName":"StationName", 
             "MonitoringLocationTypeName":"StationType"} 

In [19]:
WQPStat.rename(columns=StatFieldDict,inplace=True)

In [20]:
statdroplist = ["ContributingDrainageAreaMeasure/MeasureUnitCode", "ContributingDrainageAreaMeasure/MeasureValue", 
                "DrainageAreaMeasure/MeasureUnitCode", "DrainageAreaMeasure/MeasureValue", "CountryCode", "ProviderName", 
                "SourceMapScaleNumeric"]
WQPStat.drop(statdroplist,inplace=True,axis=1)

In [21]:
TypeDict = {"Stream: Canal":"Stream", "River/Stream":"Stream", 
            "Stream: Canal":"Stream", "Well: Test hole not completed as a well":"Well"}
WQPStat.StationType = WQPStat["StationType"].apply(lambda x: TypeDict.get(x,x),1)
WQPStat.Elev = WQPStat.Elev.apply(lambda x: np.nan if x==0.0 else round(x,1), 1)

In [22]:
WQPStat['StationId'] = WQPStat['StationId'].str.replace('_WQX-','-')
WQPStat.drop_duplicates(subset=['StationId'],inplace=True)

##SDWIS

SDWIS data were extracted from the Utah SDWIS database into ArcGIS 10.3.2 using the following SQL query.  NED 10m elevation and UTM coordinates were appended using ArcGIS.

```SQL
SELECT UTV80.TINWSF.EXTERNAL_SYS_NUM AS "FED_NM",  UTV80.TINWSF.ST_ASGN_IDENT_CD AS "ST_ID",  UTV80.TINWSF.TYPE_CODE,  UTV80.TINWSYS.NAME AS "SYS_NM", UTV80.TINWSYS.D_PRIN_CNTY_SVD_NM AS "COUNTY", UTV80.TINWSF.NAME AS "FAC_NM",  UTV80.TINWSF.TINWSYS_IS_NUMBER AS "SY_NBR", UTV80.TINLOC.LATITUDE_MEASURE AS "Y",  UTV80.TINLOC.LONGITUDE_MEASURE AS "X",  UTV80.TINLOC.VERTICAL_MEASURE AS "Z", UTV80.TSASAMPL.COLLLECTION_END_DT AS "DTE", UTV80.TSAANLYT.NAME AS "ANLY_NM",  UTV80.TSASAR.CONCENTRATION_MSR AS "CONC_MSR", UTV80.TSASAR.TSASAR_IS_NUMBER AS "ID_NUM",  UTV80.TSASAR.UOM_CODE,   UTV80.TSASAR.DETECTN_LIMIT_NUM AS "DET_LIM", UTV80.TSASAR.DETECTN_LIM_UOM_CD AS "DET_UOM"  FROM  UTV80.TINWSF  INNER JOIN  UTV80.TINWSYS ON UTV80.TINWSF.TINWSYS_IS_NUMBER = UTV80.TINWSYS.TINWSYS_IS_NUMBER  INNER JOIN  UTV80.TINLOC ON UTV80.TINWSF.TINWSF_IS_NUMBER = UTV80.TINLOC.TINWSF_IS_NUMBER  INNER JOIN  UTV80.TSASMPPT ON UTV80.TINWSF.TINWSF_IS_NUMBER = UTV80.TSASMPPT.TINWSF0IS_NUMBER  INNER JOIN  UTV80.TSASAMPL ON UTV80.TSASMPPT.TSASMPPT_IS_NUMBER = UTV80.TSASAMPL.TSASMPPT_IS_NUMBER  INNER JOIN  UTV80.TSASAR ON UTV80.TSASAMPL.TSASAMPL_IS_NUMBER = UTV80.TSASAR.TSASAMPL_IS_NUMBER  INNER JOIN  UTV80.TSAANLYT ON UTV80.TSASAR.TSAANLYT_IS_NUMBER = UTV80.TSAANLYT.TSAANLYT_IS_NUMBER WHERE  (UTV80.TINWSYS.D_PRIN_CNTY_SVD_NM LIKE '%CACHE COUNTY%')   AND (UTV80.TSAANLYT.NAME LIKE '%NITRATE%' OR UTV80.TSAANLYT.NAME LIKE '%NITRITE%' OR UTV80.TSAANLYT.NAME LIKE '%AMMONI%' OR UTV80.TSAANLYT.NAME LIKE '%SULFATE%' OR UTV80.TSAANLYT.NAME LIKE '%TDS%' OR UTV80.TSAANLYT.NAME LIKE '%SODIUM%' OR UTV80.TSAANLYT.NAME LIKE '%FLUORIDE%' OR UTV80.TSAANLYT.NAME LIKE '%MAGNESIUM%' OR UTV80.TSAANLYT.NAME LIKE '%SELENIUM%' OR  UTV80.TSAANLYT.NAME LIKE '%CALCIUM%' OR UTV80.TSAANLYT.NAME LIKE '%CHLORIDE%' OR UTV80.TSAANLYT.NAME LIKE '%POTASSIUM%' OR UTV80.TSAANLYT.NAME LIKE '%SELENIUM%' OR UTV80.TSAANLYT.NAME LIKE '%SILICA%' OR UTV80.TSAANLYT.NAME LIKE '%IRON %' OR UTV80.TSAANLYT.NAME LIKE '%ALKA %' OR UTV80.TSAANLYT.NAME LIKE '%CONDUCTIVITY%' OR UTV80.TSAANLYT.NAME LIKE '%PH %' OR UTV80.TSAANLYT.NAME LIKE '%TEMP%' OR UTV80.TSAANLYT.NAME LIKE '%ARSENIC%' OR UTV80.TSAANLYT.NAME LIKE '%CARBON%' OR UTV80.TSAANLYT.NAME LIKE '%TRITIUM%' OR UTV80.TSAANLYT.NAME LIKE '%COPPER%' OR UTV80.TSAANLYT.NAME LIKE '%LEAD%' OR UTV80.TSAANLYT.NAME LIKE '%NITROGEN%' OR UTV80.TSAANLYT.NAME LIKE '%PHOSPHATE%' OR UTV80.TSAANLYT.NAME LIKE '%TDS%' OR UTV80.TSAANLYT.NAME LIKE '%ZINC%' OR UTV80.TSAANLYT.NAME LIKE '%IRON%' OR UTV80.TSAANLYT.NAME LIKE '%CHROMIUM%' ) ORDER BY UTV80.TINWSF.ST_ASGN_IDENT_CD
```

In [23]:
SDWIS = pd.read_csv(SDWISFile)

def sampid(x):
    return "SDWIS" + str(x[0]) + str(x[1]) + str(x[2])[:-7]

def statid(x):
    return "SDWIS" + str(x[0]) + str(x[1]) 

def statnm(x):
    return str(str(x[0]) + " " + str(x[1])).title()

SDWIS["StationId"] = SDWIS[["FED_NM","ST_ID"]].apply(lambda x: statid(x),1)
SDWIS["StationName"] = SDWIS[["SYS_NM","FAC_NM"]].apply(lambda x: statnm(x),1)
SDWIS["SampleId"] = SDWIS[["FED_NM","ST_ID","DTE"]].apply(lambda x: sampid(x),1)
SDWIS["OrgId"] = "UDDW"
SDWIS["OrgName"] = "Utah Division of Drinking Water"
SDWIS["Elev"] = SDWIS["Z"].apply(lambda x: round(x*3.2808,1),1)
SDWIS["Unit"] = SDWIS["UOM_CODE"].apply(lambda x: str(x).lower(),1)
SDWIS["MDLUnit"] = SDWIS["DET_UOM"].apply(lambda x: str(x).lower(),1)
SDWIS["Param"] = SDWIS["ANLY_NM"].apply(lambda x: str(x).title().rstrip(),1)

In [24]:
SDWISFields ={"DTE":"SampleDate", "TYPE_CODE":"StationType",
              "CONC_MSR":"ResultValue", "DET_LIM":"MDL",
              "Y":"Lat_Y", "X":"Lon_X"}  
SDWIS.rename(columns=SDWISFields,inplace=True)

In [25]:
print sorted(list(SDWIS.Param.unique()))

def parnormSDWIS(x):
    p = str(x[0]).rstrip().lstrip().lower()
    u = str(x[2]).rstrip().lstrip().lower()
    if p == 'nitrate':
        return 'Nitrate', x[1]*4.427, 'mg/l'
    elif p == 'nitrite':
        return 'Nitrite', x[1]*3.285, 'mg/l'
    elif p == 'nitrogen-ammonia as (n)':
        return 'Ammonium', x[1]*1.288, 'mg/l'
    elif u == 'ug/l':
        return x[0], x[1]/1000, 'mg/l'
    else:
        return x[0], x[1], str(x[2]).rstrip()
    
SDWIS['Param'], SDWIS['ResultValue'], SDWIS['Unit'] = zip(*SDWIS[['Param','ResultValue','Unit']].apply(lambda x: parnormSDWIS(x),1))

['Alkalinity, Carbonate', 'Arsenic', 'Bicarbonate As Hco3', 'Calcium', 'Carbon Dioxide', 'Carbon Tetrachloride', 'Carbon, Total', 'Chloride', 'Chromium', 'Chromium, Hex', 'Conductivity @ 25 C Umhos/Cm', 'Copper, Free', 'Fluoride', 'Hardness, Carbonate', 'Hydroxide As Calcium Carbonate', 'Iron', 'Iron, Dissolved', 'Lead', 'Magnesium', 'Nitrate', 'Nitrate-Nitrite', 'Nitrite', 'Nitrogen-Ammonia As (N)', 'Orthophosphate', 'Ph', 'Phosphate, Total', 'Potassium', 'Selenium', 'Silica', 'Sodium', 'Sulfate', 'Tds', 'Tritium', 'Vinyl Chloride', 'Zinc']


In [26]:
SDWIS.drop(["FED_NM", "DET_UOM", "UOM_CODE","ANLY_NM", "FAC_NM", "ST_ID", 
            "SYS_NM", "COUNTY", "SY_NBR", "Z", "ID_NUM"],inplace=True, axis=1)

In [27]:
SDWISPmatch = {"Ph":"pH","Tds":"TDS","Nitrogen-Ammonia As (N)":"Nitrogen-Ammonia as (N)",
              "Hydroxide As Calcium Carbonate":"Hydroxide as Calcium Carbonate",
               "Bicarbonate As Hco3":"Bicarbonate as HCO3"}
SDWIS["Param"] = SDWIS["Param"].apply(lambda x: SDWISPmatch.get(x,x))
SDWIS["StationName"] = SDWIS["StationName"].apply(lambda x: x.replace("Wtp","WTP"))
#SDWIS["Unit"] = SDWIS["Unit"].apply(lambda x: x.rstrip(),1)
SDWIS["ResultValue"] = SDWIS[["ResultValue","Unit"]].apply(lambda x: x[0]/1000 if x[1]=="ug/L" else x[0], 1)
#SDWIS["Unit"] = SDWIS["Unit"].apply(lambda x: unitfix(x),1)

In [28]:
SDWISType = {"SP":"Spring","WL":"Well","TP":"Facility Other","IN":"Stream","CC":"Connection"}
SDWIS.StationType = SDWIS.StationType.apply(lambda x: SDWISType.get(x,x),1)

SDWIS facility type code (FacTypeCode): CC, Consecutive_connection;
CH, Common_headers; CS, Cistern; CW, Clear_well;
DS, Distribution_system_zone; IG, Infiltration_gallery; IN,
Intake; NP, Non-piped; OT, Other; PC, Pressure_control;
PF, Pump_facility; RC, Roof_catchment; RS, Reservoir; SI,
Surface_impoundment; SP, Spring; SS, Sampling_station; ST,
Storage; TM, Transmission_main; TP, Treatment_plant; WH,
Well_head; WL, Well.

In [29]:
SDWISSta = SDWIS.drop([u'SampleDate', u'ResultValue', u'MDL', u'SampleId', u'Unit', u'MDLUnit', u'Param'], axis=1)
SDWISSta.drop_duplicates(inplace=True)

In [30]:
SDWISRes = SDWIS.drop([u'StationType', u'Lat_Y', u'Lon_X', u'StationName', u'Elev'], axis=1)

##UDAF

In [31]:
AGStat = pd.read_csv(AGStationsFile)
AGStat["StationType"] = "Well"
AGStatFields = {"SITEID":"StationId","FINISHEDDE":"Depth","POINT_Y":"Lat_Y",
                "POINT_X":"Lon_X","ELEV_FT":"Elev","ACCURACY":"HorAcc"}
AGStat.rename(columns=AGStatFields,inplace=True)

In [32]:
AGStat.drop(["OBJECTID_1", "OBJECTID", "PUB_YR", "SAMPLENO", "WLDATE", "WLDEPTH"], inplace=True, axis=1)

In [33]:
AGStat.StationId = AGStat.StationId.apply(lambda x: "UDAF-"+str(int(x)).zfill(5),1)

In [34]:
names = ["SampleId","ResultValue", "ParAbb", "Unit", "Param", "MDL","BelowLim","TestNo",
         "StationId","SampleDate","SampYear"]
AGRes = pd.read_csv(AGResultsFile, names=names, index_col=10)

In [35]:
AGRes["DetectCond"] = AGRes["BelowLim"].apply(lambda x: 'Not Detected' if x=='Y' else np.nan,1)

In [36]:
AGRes.ResultValue = AGRes[["BelowLim","ResultValue"]].apply(lambda x: np.nan if x[0]=="Y" or x[1] == 0.0 else x[1], 1)

In [37]:
def parnormAG(x):
    p = str(x[0]).rstrip().lstrip().lower()
    u = str(x[2]).rstrip().lstrip().lower()
    if p == 'nitrate-n':
        return 'Nitrate', x[1]*4.427, 'mg/l'
    elif u == 'ug/l':
        return x[0], x[1]/1000, 'mg/l'
    else:
        return x[0], x[1], str(x[2]).rstrip()
    
AGRes['Param'], AGRes['ResultValue'], AGRes['Unit'] = zip(*AGRes[['Param','ResultValue','Unit']].apply(lambda x: parnormAG(x),1))

In [38]:
AGRes.Unit.unique()

array(['mg/L', 'mg/l', 'meq/L', '\xb0C', '--', 'mS/cm', 'nan'], dtype=object)

In [39]:
AGRes.dropna(subset=["StationId","ResultValue"], how="any", inplace=True)

In [40]:
AGRes.StationId = AGRes.StationId.apply(lambda x: "UDAF-"+str(int(x)).zfill(5),1)

In [41]:
AGStAv = list(AGStat.StationId.values)
AGRes = AGRes[AGRes.StationId.isin(AGStAv)]

In [42]:
AGStat['OrgId']='UDAF'

##STORET Legacy

In [43]:
STORLegSta = pd.read_table(STORLegStatFile, skiprows=[1])
STORLegRes = pd.read_table(STORLegResFile, skiprows=[1])
STORParam = pd.read_table(STORParamFile)

In [44]:
rescol = list(STORLegRes.columns)
j = []
for i in rescol:
    j.append(i.rstrip("\t").rstrip().lstrip().replace(" ",""))
resdict = dict(zip(rescol,j))
STORLegRes.rename(columns=resdict,inplace=True)

statcol = list(STORLegSta.columns)
k = []
for i in statcol:
    k.append(i.rstrip("\t").rstrip().lstrip().replace(" ",""))
statdict = dict(zip(statcol,k))
STORLegSta.rename(columns=statdict,inplace=True)

In [45]:
STORLegRes["SampleDate"] = STORLegRes[["StartDate","StartTime"]].apply(lambda x: datetimefix(x,"%Y-%m-%d %H:%M"),1)
STORLegRes = STORLegRes[STORLegRes.SecondaryActivityCategory.isin(['Water',np.nan])]
STORParamDict = dict(zip(STORParam['Parameter No.'].values, STORParam['Full Name'].values))
STORLegRes.Param = STORLegRes.Param.apply(lambda x: STORParamDict.get(x),1) 

In [46]:
STORResField = {"Agency":"OrgId","AgencyName":"OrgName","Station":"StationId","SampleDepth":"SampDepth"}
STORLegRes.rename(columns=STORResField,inplace=True)

In [47]:
STORLegRes.drop(["StateName", "CountyName", "HUC", "EndDate", "UMK", "CS", "ReplicateNumber",
                 "COMPOSITE_GRAB_NUMBER","CM","PrimaryActivityCategory","PrimaryActivityCategory",
                 "SecondaryActivityCategory",
                 "EndTime", "StartDate", "StartTime", "Latitude", "Longitude"],inplace=True,axis=1)

In [48]:
STORLegRes["SampleId"] = STORLegRes[["StationId","SampleDate"]].apply(lambda x: str(x[0]) + "-" + str(x[1]),1 )
STORLegRes["StationId"] = STORLegRes["StationId"].apply(lambda x: "EPALeg-" + x, 1)

In [49]:
STORLegRes.Param = STORLegRes.Param.apply(lambda x: str(x).title(),1)

In [50]:
STORLegRes.columns

Index([u'OrgId', u'StationId', u'StationName', u'OrgName', u'ResultValue',
       u'R', u'Param', u'SampDepth', u'SampleDate', u'SampleId'],
      dtype='object')

In [51]:
def parnormSTOR(x):
    p = str(x[0]).rstrip().lstrip().lower()
    if p ==  'nitrate nitrogen, total (mg/L as n)' or p== 'nitrate nitrogen, total':
        return 'Nitrate', x[1]*4.427, 'mg/l'
    elif p == 'nitrite nitrogen, total (mg/l as n)':
        return 'Nitrite', x[1]*3.285, 'mg/l'
    elif p == 'nitrogen, ammonia, total (mg/l as n)':
        return 'Ammonium', x[1]*1.288, 'mg/l'
    elif p == 'sulfate (as s) whole     water, mg/L':
        return 'Sulfate', x[1]*2.996, 'mg/l'
    elif p in ('phosphorus, dissolved orthophosphate (mg/l as p)'):
        return 'Phosphate', x[1]*3.066, 'mg/l'
    else:
        return x[0], x[1], np.nan

STORLegRes['Param'], STORLegRes['ResultValue'], STORLegRes['Unit'] = zip(*STORLegRes[['Param','ResultValue']].apply(lambda x: parnormSTOR(x),1))

In [52]:
STORKeepers = ['Temperature, Water (Degrees Centigrade)',
               'Temperature, Water (Degrees Fahrenheit)', 
               'Specific Conductance,Field (Umhos/Cm @ 25C)',
               'Specific Conductance (Umhos/Cm @ 25C)', 
               'Sulfate (As S) Whole     Water, Mg/L',
               'Oxygen, Dissolved                             Mg/L',
               'Oxygen, Dissolved, Percent Of Saturation         %',
               'Bod, 5 Day, 20 Deg C                          Mg/L',
               'Ph (Standard Units)', 'Ph, Lab, Standard Units                         Su',
               'Carbon Dioxide (Mg/L As Co2)', 'Alkalinity,Total,Low Level Gran Analysis     Ueq/L',
               'Alkalinity, Total (Mg/L As Caco3)', 'Bicarbonate Ion (Mg/L As Hco3)', 'Carbonate Ion (Mg/L As Co3)',
               'Nitrogen, Ammonia, Total (Mg/L As N)', 'Ammonia, Unionzed                      (Mg/L As N)',
               'Nitrite Nitrogen, Total (Mg/L As N)', 'Ammonia, Unionized (Calc Fr Temp-Ph-Nh4)  (Mg/L)',
               'Nitrate Nitrogen, Total (Mg/L As N)', 'Nitrogen, Kjeldahl, Total, (Mg/L As N)',
               'Nitrite Plus Nitrate, Total 1 Det. (Mg/L As N)', 'Phosphorus (P), Water, Total Recoverable      Ug/L',
               'Phosphorus, Total (Mg/L As P)', 'Phosphorus, Dissolved Orthophosphate (Mg/L As P)',
               'Carbon, Dissolved Organic (Mg/L As C)',
               'Carbon, Dissolved Inorganic (Mg/L As C)',
               'Hardness, Total (Mg/L As Caco3)', 'Calcium (Mg/L As Caco3)',
               'Calcium, Dissolved (Mg/L As Ca)',
               'Magnesium, Dissolved (Mg/L As Mg)',
               'Sodium, Dissolved (Mg/L As Na)',
               'Potassium, Dissolved (Mg/L As K)',
               'Chloride, Dissolved In Water           Mg/L',
               'Sulfate, Dissolved (Mg/L As So4)',
               'Fluoride, Dissolved (Mg/L As F)',
               'Silica, Dissolved (Mg/L As Si02)',
               'Arsenic, Dissolved  (Ug/L As As)', 'Arsenic, Total (Ug/L As As)',
               'Barium, Dissolved (Ug/L As Ba)', 'Barium, Total (Ug/L As Ba)',
               'Beryllium, Total (Ug/L As Be)', 'Boron, Dissolved (Ug/L As B)',
               'Boron, Total (Ug/L As B)', 'Cadmium, Dissolved (Ug/L As Cd)',
               'Cadmium, Total (Ug/L As Cd)', 'Chromium, Dissolved (Ug/L As Cr)',
               'Chromium, Hexavalent (Ug/L As Cr)', 'Chromium, Total (Ug/L As Cr)',
               'Copper, Dissolved (Ug/L As Cu)', 'Copper, Total (Ug/L As Cu)',
               'Iron, Dissolved (Ug/L As Fe)', 'Lead, Dissolved (Ug/L As Pb)',
               'Lead, Total (Ug/L As Pb)', 'Manganese, Total (Ug/L As Mn)',
               'Manganese, Dissolved (Ug/L As Mn)', 'Thallium, Total (Ug/L As Tl)',
               'Nickel, Dissolved (Ug/L As Ni)', 'Nickel, Total (Ug/L As Ni)',
               'Silver, Dissolved (Ug/L As Ag)', 'Silver, Total (Ug/L As Ag)',
               'Zinc, Dissolved (Ug/L As Zn)', 'Zinc, Total (Ug/L As Zn)',
               'Antimony, Total (Ug/L As Sb)', 'Aluminum, Total (Ug/L As Al)',
               'Selenium, Dissolved (Ug/L As Se)', 'Selenium, Total (Ug/L As Se)',
               'Tritium (1H3),Total (Picocuries/Liter)',
               'Hardness, Ca Mg Calculated (Mg/L As Caco3)',
               'Chlorine, Total Residual (Mg/L)',
               'Residue,Total Filtrable (Dried At 180C),Mg/L',
               'Nitrate Nitrogen, Dissolved (Mg/L As No3)', 'Iron (Ug/L As Fe)',
               'Phosphorus, Total, As Po4 - Mg/L', 'Mercury, Total  (Ug/L As Hg)']
STORLegRes = STORLegRes[STORLegRes.Param.isin(STORKeepers)] 

In [53]:
def parsplit(x,p):
    x = str(x).rstrip().lstrip()
    if p == "Un":
        z = -1
        x = str(x).replace("Mg/L", "mg/l")
        x = str(x).replace("Ug/L", "ug/l")
        x = str(x).replace("o", "O")
        x = str(x).replace("c", "C")
        x = str(x).replace("TOtal ReCOverable","Total Recoverable")        
        x = str(x).replace("UmhOs", "umhos")
        x = str(x).replace("TOtal","Total")
    elif p== "Par":
        z = 0
        x = str(x).replace(", Standard Units","")
        x = str(x).replace(", Unionized","")
        x = str(x).replace(", Unionzed","")
        x = str(x).replace(",Low Level Gran Analysis","")
        x = str(x).replace(" Ion","")
        x = str(x).replace(",Total",", Total")
        if x == "Ph" or x == "Ph, Lab":
            x = str(x).replace("Ph","pH")
    if "(" in x:
        x = str(x).replace(" As ", " as ")
        return str(x).split(" (")[z].rstrip(")").rstrip().lstrip()
    else:
        return str(x).split("  ")[z].rstrip().lstrip()

def splitmore(x):
    if "NO3" in x:
        return x
    elif " as " in x:
        return x.split(" as ")[0]
    elif x == "As S) WhOle     Water, mg/l" or x == "Dried At 180C),mg/l" or x=="PhOsphOrus, Total, As PO4 - mg/l":
        return "mg/l"
    elif x == "P), Water, Total Recoverable      ug/l":
        return "ug/l"
    else:
        return x

def unitconv(x):
    if x[1]=="ug/l":
        return x[0]/1000
    elif x[1]=="Degrees Fahrenheit":
        return (float(x[0])-32.0)*(5.0/9.0)
    else:
        return x[0]

In [54]:
STORLegRes["Unit"] = STORLegRes["Param"].apply(lambda x: parsplit(x,"Un"), 1)
STORLegRes["Param"] = STORLegRes["Param"].apply(lambda x: parsplit(x,"Par"), 1)
STORLegRes["Unit"] = STORLegRes["Unit"].apply(lambda x: splitmore(x), 1)
STORLegRes["ResultValue"] = STORLegRes[["ResultValue","Unit"]].apply(lambda x: unitconv(x), 1)
STORLegRes["Unit"] = STORLegRes["Unit"].apply(lambda x: "mg/l" if x=="ug/l" else x, 1)
STORLegRes["Unit"] = STORLegRes["Unit"].apply(lambda x: "Degrees Centigrade" if x=="Degrees Fahrenheit" else x, 1)

In [55]:
STORStaField = {"Agency":"OrgId","AgencyName":"OrgName","Station":"StationId", "DepthUnits":"DepthUnit",
                "Latitude":"Lat_Y", "Longitude":"Lon_X", "HUC":"HUC8", "StationDepth":"Depth"}
STORLegSta.rename(columns=STORStaField,inplace=True)

In [56]:
STORLegSta.columns

Index([u'OrgId', u'StationId', u'StationName', u'OrgName', u'StateName',
       u'CountyName', u'Lat_Y', u'Lon_X', u'HUC8', u'RchmileSegment',
       u'MilesUpReach', u'Rchonoff', u'Rchname', u'StationAlias',
       u'StationType', u'Depth', u'DepthUnit', u'S', u'G', u'Description'],
      dtype='object')

In [57]:
STORLegSta.drop(["RchmileSegment", "MilesUpReach", "Rchonoff", "Description", "G", "S", "StationAlias",
                 "Rchname", "StateName", "CountyName"], inplace=True, axis=1)

In [58]:
STORLegSta.StationType = STORLegSta.StationType.apply(lambda x: str(x).rstrip(" ").strip("/SUPPLY").split("/")[-1].title(),1)

In [59]:
LegTypeDict = {"We":"Well"}
STORLegSta.StationType = STORLegSta.StationType.apply(lambda x: LegTypeDict.get(x,x),1)

In [60]:
STORLegSta.StationId = STORLegSta["StationId"].apply(lambda x: "EPALeg-" + x, 1)

##UGS Data

In [61]:
UGSfield = pd.read_excel(fielddata,"FieldChem")
UGSNO3 = pd.read_excel(fielddata,"Nitrate")

In [62]:
UGS = pd.read_csv(UGSFile, engine="python")
UGS["StationId"] = UGS["SITE"].apply(lambda x:"UGS-"+str(x).zfill(4),1) 

In [63]:
UGSSta = UGS.drop([u'OBJECTID_1',u'SITE', u'TDS', u'Temp', u'Cond', u'CO2', u'HCO3', 
                  u'CO3',u'Na', u'pH', u'Ca', u'SO4', u'NO3', u'As_', u'Cl', u'K',
                  u'Mg', u'Hard', u'NH4'], axis=1)

In [64]:
UGSRe = UGS.drop([u'OBJECTID_1',u'SITE',u'StationType', u'Geology', u'Elev', u'Lat_Y', u'Lon_X', u'StationName', 
                  u'OrgId', u'WRNUM', u'SITE', u'UTM_X', u'UTM_Y', u'Depth_ft'], axis=1)

In [65]:
UGSRe["SampleId"] = UGSRe.index

In [66]:
UGSRe.reset_index(inplace=True)
UGSRe.set_index(["StationId","SampleId"], inplace=True)

In [67]:
UGSRe.drop(UGSRe.columns[0],inplace=True,axis=1)

In [68]:
UGSStack = UGSRe.stack().to_frame()
UGSStack.columns = ["ResultValue"]

In [69]:
UGSStack.reset_index(inplace=True)

In [70]:
UGSStack.columns=["StationId","SampleId","Param","ResultValue"]

In [71]:
def unitcon(x):
    if x=="pH":
        return ""
    elif x=="Temp":
        return "C"
    elif x=="Cond":
        return "uS/cm"
    else:
        return "mg/l"
    
UGSStack["Unit"] = UGSStack["Param"].apply(lambda x: unitcon(x),1)
UGSStack["ParAbb"] = UGSStack["Param"]
UGSStack["OrgId"] = "UGS"
UGSStack["OrgName"] = "Utah Geological Survey"
UGSStack["ResultValue"] = UGSStack[['Param','ResultValue']].apply(lambda x: x[1]*1.288 if x[0]=='Ammonia as N' else x[1],1)
UGSStack["Param"] = UGSStack['Param'].apply(lambda x: 'Ammonia' if x=='Ammonia as N' else x, 1)
UGSStack["ResultValue"] = UGSStack[['Param','ResultValue']].apply(lambda x: x[1]*3.066 if x[0]=='Phosphate, Tot. Dig. (as P)' else x[1],1)
UGSStack["Param"] = UGSStack['Param'].apply(lambda x: 'Phosphate' if x=='Phosphate, Tot. Dig. (as P)' else x, 1)

##State Lab

In [74]:
SLSampMatch = pd.read_excel(fielddata,"StateLabMatch")
SLStations = pd.read_excel(fielddata,"Stations")
SLStat = pd.merge(SLSampMatch, SLStations, on='StationId', how='left')
#SLStat.reset_index(inplace=True)

In [75]:
SL0 = pd.read_table(statelabresults0, sep="\t", lineterminator="\n", error_bad_lines=False)
SL0 = SL0[SL0['Collector']=='PI']
SL1 = pd.read_table(statelabresults1, sep="\t", lineterminator="\n", error_bad_lines=False)
SL1 = SL1[SL1['Collector']=='PI']
SL2 = pd.read_table(statelabresults2, sep="\t", lineterminator="\n", error_bad_lines=False)
SL2 = SL2[SL2['Collector']=='PI']

Skipping line 11: expected 43 fields, saw 44
Skipping line 19: expected 43 fields, saw 44
Skipping line 238: expected 43 fields, saw 44
Skipping line 857: expected 43 fields, saw 44

Skipping line 726: expected 43 fields, saw 44
Skipping line 745: expected 43 fields, saw 44
Skipping line 1467: expected 43 fields, saw 44
Skipping line 1487: expected 43 fields, saw 44
Skipping line 1507: expected 43 fields, saw 44
Skipping line 1527: expected 43 fields, saw 44



In [76]:
SL = pd.concat([SL0,SL1,SL2])

In [77]:
SL["OrgId"] = "UGS"
SL["OrgName"] = "Utah Geological Survey"
SL['DetectCond'] = SL['Problem#Identifier'].apply(lambda x: 'Not Detected' if str(x).rstrip()=='<' else np.nan,1)
SL['SampleDate'] = SL[['Sample#Date','Sample#Time']].apply(lambda x: datetimefix(x,"%m/%d/%y %H:%M"),1)
SLHead = {'Sample#Number':'SampleId', 'Param#Description':'Param', 'Result#Value':'ResultValue','Units':'Unit', 
          'Lower#Report#Limit':'MDL','Method#ID':'SampMeth','Analysis#Date':'AnalysisDate'}
SL.rename(columns=SLHead,inplace=True)

In [78]:
SL.drop([u'Lab#Code', u'Station#ID', u'Source#Code', u'Sample#Date',
       u'Sample#Time', u'Sample#Type', u'Cost#Code', u'Billing#Code',
       u'Agency#Bill#Code', u'Trip#ID', u'Sample#Description', u'Collector',
       u'Sample#Recieved#Date', u'Chain#of#Custody#Ind.', u'Replicate#Number',
       u'Sample#Comment', u'Method#Number', u'Method#Agency',
       u'Method#Description', u'Param#Number', u'CAS#Number',
       u'Matrix#Number', u'Matrix#Description', u'Preparation#Date',
       u'Problem#Identifier', u'Result#Code', 
       u'Upper#Quant#Limit', u'Method#Detect#Limit',
       u'Confidence#Limit', u'%#Confidence#Limit',u'Dilution#Factor',
       u'Batch#Number',u'Comment#Number', u'Comment#Text'], inplace=True, axis=1)

In [79]:
SL.columns

Index([u'AnalysisDate', u'Comment#Number.1', u'Comment#Text.1', u'MDL',
       u'SampMeth', u'Param', u'ResultValue', u'SampleId', u'Unit', u'OrgId',
       u'OrgName', u'DetectCond', u'SampleDate'],
      dtype='object')

In [80]:
SLRes = pd.merge(SL, SLStat, on='SampleId', how='left')

In [81]:
SLRes.drop([u'StationName', u'Elev', u'Lat_Y', u'Lon_X', u'Depth', 
            u'DepthUnit',  u'ScreenDepth', u'WIN', u'Diameter', u'WL', u'WLDate', u'ScreenBottom'], inplace=True, axis=1)

In [82]:
SLStat.drop(['SampleId'],inplace=True, axis=1)
SLStat.drop_duplicates(inplace=True)

In [83]:
def SLparnorm(x):
    p = str(x[0]).rstrip().lstrip().lower()
    u = str(x[2]).rstrip().lstrip().lower()
    if p == 'nitrate nitrogen, total (mg/l as n)':
        return 'Nitrate', x[1]*4.427, 'mg/l'
    elif p == 'nitrite nitrogen, total (mg/l as n)':
        return 'Nitrite', x[1]*3.285, 'mg/l'
    elif p == 'ammonia as n':
        return 'Ammonium', x[1]*1.288, 'mg/l'
    elif p == 'sulfate (as s) whole     water, mg/L':
        return 'Sulfate', x[1]*2.996, 'mg/l'
    elif p in ('phosphate, tot. dig. (as p)', 'phosphate-phosphorus as p','orthophosphate as p'):
        return 'Phosphate', x[1]*3.066, 'mg/l'
    elif u == 'ug/l':
        return x[0], x[1]/1000, 'mg/l'
    else:
        return x[0], x[1], str(x[2]).rstrip()

def MDLfix(x):
    u = str(x[1]).rstrip().lstrip().lower()
    if np.isfinite(x[2]):
        return x[0]
    elif u=='ug/l':
        return x[0]/1000
    else:
        return x[0]


SLRes['MDL'] = SLRes[['MDL','Unit','ResultValue']].apply(lambda x: MDLfix(x),1)
SLRes['Param'], SLRes['ResultValue'], SLRes['Unit'] = zip(*SLRes[['Param','ResultValue','Unit']].apply(lambda x: parnorm(x),1))

#Combine Data

In [84]:
Res = pd.concat([STORLegRes,AGRes,SDWISRes,WQP,UGSStack,SLRes,UGSfield,UGSNO3])

In [85]:
Res = Res[~Res["Unit"].isin(['ueq/L','Ueq/L','ueq/l','tons/ac ft','tons/day','meq/L'])]
Res = Res[~Res["Param"].isin(["Heptachlorobiphenyl", "Hydrocarbons", "Hydroxide", "Ionic strength",
                              "Floating debris, severity", "Carbon Tetrachloride", "Trichlorobiphenyl",
                              "Vinyl Chloride", "True color", "Color", "Trash, Debris, Floatables",
                              "Total volatile solids", "Temperature, air", "Residue, Total Filtrable",
                              "Pentachlorobiphenyl", "Odor threshold number", "Odor, atmospheric",
                              "Instream features, est. stream width", "Hydroxide", 
                              "Light, transmissivity","Algae, floating mats (severity)"])]
len(Res)

436447

In [86]:
Res[["Param","Unit","USGSPCode"]].drop_duplicates(subset=["Param","Unit"]).sort(["Param"]).to_clipboard()

In [87]:
Stat = pd.concat([STORLegSta, AGStat, SDWISSta, WQPStat, SLStat, UGSSta])

In [88]:
parmatch = pd.read_excel(rootname + "Aquachem.xlsx")

In [89]:
parmatch.columns

Index([u'Param', u'Unit', u'ParrAbb'], dtype='object')

In [90]:
parmatchdict = dict(zip(parmatch.Param.values, parmatch.ParrAbb.values))
Res["ParAbb"] = Res[["ParAbb","Param"]].apply(lambda x: parmatchdict.get(x[1],x[0]),1)

In [91]:
results = Res.dropna(subset=["StationId","Param","SampleId"], how="any")

In [92]:
Stat["StationName"] = Stat["StationName"].apply(lambda x: str(x).strip().lstrip().rstrip(),1)
results["Unit"] = results[["ParAbb","Unit"]].apply(lambda x: "C" if x[0]=="Temp" else x[1],1)
results["Unit"] = results[["ParAbb","Unit"]].apply(lambda x: "umhos/cm" if x[0]=="Cond" else x[1],1)
results["Unit"] = results[["ParAbb","Unit"]].apply(lambda x: "" if x[0]=="pH" else x[1],1)

C:\Python27\ArcGIS10.3\lib\site-packages\IPython\kernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app
C:\Python27\ArcGIS10.3\lib\site-packages\IPython\kernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Python27\ArcGIS10.3\lib\site-packages\IPython\kernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the th

In [93]:
results.drop(["AnalysisDate","AnalytMeth","SampType","AnalytMethId", "BelowLim", "StationName",
              "MethodDescript", "LabComments", "LabName", "LimitType", "ProjectId", "QualCode",
              "OrgName","R", "ResultComment","ResultStatus","SampComment", "SampEquip", 
              "SampDepthRef", "SampDepthU","SampDepth","SampType", "USGSPCode",
              "SampMeth", "SampMethName","SampYear","TestNo"],inplace=True,axis=1)

C:\Python27\ArcGIS10.3\lib\site-packages\IPython\kernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [94]:
results.columns

Index([u'Comment#Number.1',   u'Comment#Text.1',       u'DetectCond',
                    u'MDL',          u'MDLUnit',            u'OrgId',
                 u'ParAbb',            u'Param',      u'ResultValue',
               u'SampFrac',        u'SampMedia',       u'SampleDate',
               u'SampleId',        u'StationId',      u'StationType',
                   u'Unit'],
      dtype='object')

In [95]:
aquc = results[["Param","OrgId","Unit","ParAbb"]]
aquc.drop_duplicates(["Param","Unit"],inplace=True)
aquc.sort(["Param"],inplace=True)
aquc.to_clipboard()

C:\Python27\ArcGIS10.3\lib\site-packages\pandas\util\decorators.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return func(*args, **kwargs)
C:\Python27\ArcGIS10.3\lib\site-packages\pandas\core\frame.py:2915: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  inplace=inplace, kind=kind, na_position=na_position)


In [96]:
def estsize(df):
    return df.values.nbytes + df.index.nbytes + df.columns.nbytes

In [97]:
(str(round(estsize(results)/1E6,2))+" MB")
#(str(round(estsize(Stat)/1E6,2))+" MB")

'31.41 MB'

In [98]:
GWStat = Stat[Stat['StationType'].isin(['Spring','Well'])]
GWStatList = list(GWStat.StationId.unique())

In [99]:
NDs = {'Not Detected':'<', 'Present Above Quantification Limit':'<', 'ND             ':'<', '*Present >QL   ':'>',
       'Present Below Quantification Limit':'<', '*Non-detect    ':'<', 'Detected Not Quantified':'<', 
       'Systematic Contamination':'<'}
results.DetectCond = results.DetectCond.apply(lambda x: NDs.get(x,np.nan),1) 

C:\Python27\ArcGIS10.3\lib\site-packages\pandas\core\generic.py:2177: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [100]:
GWResNoND = results[(results['StationId'].isin(GWStatList))] 
GWResNoND = GWResNoND[(~GWResNoND['DetectCond'].isin(['<','>']))]

In [101]:
def is_nan(x): 
    try: 
        return math.isnan(x) 
    except: 
        return False 

def detected(x):
    if x[1]=='<' and np.isfinite(x[0]):
        return x[1]+str(x[0])       
    elif x[1]=='<' and np.isfinite(x[2]):
        if str(x[3]).rstrip().lower() == 'ug/l':
            return x[1]+str(x[2]/1000)
        else:
            return x[1]+str(x[2])
    else:
        return x[0]

results.ResultValue = results[['ResultValue','DetectCond','MDL','MDLUnit']].apply(lambda x: detected(x),1)

In [148]:
GWRes = results[results['StationId'].isin(GWStatList)]


In [149]:
GWRes.drop_duplicates(subset = ['SampleId','ParAbb'],inplace=True)
GWResNoND.drop_duplicates(subset = ['SampleId','ParAbb'],inplace=True)

#Pivot Data

In [104]:
datap = GWRes.pivot(index='SampleId', columns='ParAbb', values='ResultValue')
datapND = GWResNoND.pivot(index='SampleId', columns='ParAbb', values='ResultValue')

In [105]:
datap.dropna(subset=['SO4','Cond','Temp','TDS','pH_field'],how='all',inplace=True)
datap.drop(datap.columns[[0]], axis=1, inplace=True)
datapND.dropna(subset=['SO4','Cond','Temp','TDS','pH_field'],how='all',inplace=True)
datapND.drop(datapND.columns[[0]], axis=1, inplace=True)

In [106]:
resdrop = [ 'DetectCond',
             'MDL', 'MDLUnit',
             'OrgId', 'Param',
             'ResultValue', 'SampFrac',
             'SampMedia', 
             'Unit', 'ParAbb']
resPivot = GWRes.drop(resdrop, axis=1)

In [107]:
datapiv = pd.merge(datap, resPivot, left_index=True, right_on='SampleId',how='left')
datapivND = pd.merge(datapND, resPivot, left_index=True, right_on='SampleId',how='left')

In [108]:
pivStats = GWStat.drop(['Aquifer', 'ConstDate', 'Depth', 'DepthUnit','AquiferType', 'OrgName', 'HorCollMeth',
                         'HoleDUnit', 'HoleDepth', 'HUC8', 'HorAccUnit',
                         'ElevUnit', 'ElevRef', 'ElevAcc', 'ElevMeth','CountyCode', 'ElevAccUnit',
                         'HorAcc', 'StateCode', 'HorRef',
                         'OrgId', 'StationComment', 'StationName'], axis=1)

In [109]:
pivStats.reset_index(inplace=True)
pivStats.set_index("StationId",inplace=True)

In [110]:
datapiv.drop_duplicates(inplace=True)
datapivND.drop_duplicates(inplace=True)

In [111]:
pivdata = pd.merge(datapiv, pivStats, left_on="StationId", right_index=True, how='left')
pivdataND = pd.merge(datapivND, pivStats, left_on="StationId", right_index=True, how='left')

In [112]:
pivdata.drop_duplicates(inplace=True)
pivdataND.drop_duplicates(inplace=True)

In [113]:
piperdata = pivdata.dropna(subset = ['Ca','Na','Cl','Mg','SO4'],how='any')
piperdataND = pivdataND.dropna(subset = ['Ca','Na','Cl','Mg','SO4'],how='any')

In [114]:
piperdata.drop_duplicates(subset=['SampleId'],inplace=True)
piperdataND.drop_duplicates(subset=['SampleId'],inplace=True)

In [115]:
(str(round(estsize(piperdata)/1E6,2))+" MB")

'0.4 MB'

In [116]:
GWStat.drop(['Aquifer','AquiferType','CountyCode','DepthUnit','ElevAcc','ElevAccUnit',
             'ElevMeth','ElevRef','ElevUnit','HoleDUnit','HorAcc','HorAccUnit','HorCollMeth',
            'HorRef','StateCode'],inplace=True)

C:\Python27\ArcGIS10.3\lib\site-packages\IPython\kernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [117]:
def proj(x):
    inProj = Proj(init='epsg:4326') #WGS84
    outProj = Proj(init='epsg:2152') #NAD83(CSRS98) / UTM zone 12N
    x2,y2 = transform(inProj,outProj,x[0],x[1])
    return x2, y2

GWStat['UTM_X'], GWStat['UTM_Y'] = zip(*GWStat[['Lon_X','Lat_Y']].apply(lambda x: proj(x),1))
piperdata['UTM_X'], piperdata['UTM_Y'] = zip(*piperdata[['Lon_X','Lat_Y']].apply(lambda x: proj(x),1))

C:\Python27\ArcGIS10.3\lib\site-packages\IPython\kernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Python27\ArcGIS10.3\lib\site-packages\IPython\kernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [118]:
def getelev(x):
    elev = "http://ned.usgs.gov/epqs/pqs.php?x="+str(x[0])+"&y="+str(x[1])+"&units=Meters&output=xml"
    response = urllib2.urlopen(elev)
    html = response.read()
    d = xmltodict.parse(html)
    return float(d['USGS_Elevation_Point_Query_Service']['Elevation_Query']['Elevation'])

GWStat['Elev'] = GWStat[['Lon_X','Lat_Y']].apply(lambda x: getelev(x),1)

C:\Python27\ArcGIS10.3\lib\site-packages\IPython\kernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [119]:
def winmatch(x):
    request = mechanize.Request("http://waterrights.utah.gov/wellinfo/wellsearch.asp")
    response = mechanize.urlopen(request)
    forms = mechanize.ParseResponse(response, backwards_compat=False)
    response.close()
    form = forms[0] 
    #print form
    form["mainoption"]=["radius"]
    form["SearchRadius"]="2000"
    form["option"]=["UTM"]
    form["xUTM"]=str(x[0])
    form["yUTM"]=str(x[1])
    if np.isfinite(x[2]):
        form["DateRange"]=["on"]
        form["edtDrillBef"]=str(int(str(x[2])[:4])+5)
        form["edtDrillAft"]=str(int(str(x[2])[:4])-5)
        #print str(int(str(x[2])[:4]))
    if np.isfinite(x[3]):
        form["DepthRange"]=["on"]
        form["edtDrillBel"]=str(int(x[3])-10)
        form["edtDrillAbo"]=str(int(x[3])+30)
        #print str(str(int(x[3])))
    win =  mechanize.urlopen(form.click()).read()
    winbeg = win.find('WIN=')
    if winbeg == -1:
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan 
    else:
        wintabeg=win.find('<table',win.find('<table')+5)
        wintaend=win.find('</table>')
        winmatches = pd.read_html(win[wintabeg:wintaend], header=0, skiprows=0)
        winmatches = winmatches[0]
        winDic = {u'WRNUM/Appl. No.':'WRNUM',u'Distance From Point (ft)':'DIST',u'Diameter':'Diam',u'Depth':'TD',
          u'Drilled Date':'DrillDate',u'Location(link to Log)':'Locatio',u'WIN':'WIN',u'Geologic Log':'Log'}
        winmatches.rename(columns=winDic,inplace=True)
        return winmatches.ix[0,0], winmatches.ix[0,1], winmatches.ix[0,2], winmatches.ix[0,3], winmatches.ix[0,4], winmatches.ix[0,5], int(winmatches.ix[0,6])

In [120]:
GWStat.ScreenDepth = GWStat[['ScreenDepth','SCREENDEPT']].apply(lambda x: x[0] if np.isfinite(x[0]) else x[1],1)

In [121]:
GWStat.drop_duplicates(['StationId'],inplace=True)

In [122]:
GWStat.columns

Index([       u'Aquifer',    u'AquiferType',      u'ConstDate',
           u'CountyCode',          u'Depth',      u'DepthUnit',
             u'Depth_ft',       u'Diameter',           u'Elev',
              u'ElevAcc',    u'ElevAccUnit',       u'ElevMeth',
              u'ElevRef',       u'ElevUnit',         u'FmType',
              u'Geology',           u'HUC8',      u'HoleDUnit',
            u'HoleDepth',         u'HorAcc',     u'HorAccUnit',
          u'HorCollMeth',         u'HorRef',          u'Lat_Y',
                u'Lon_X',          u'OrgId',        u'OrgName',
           u'SCREENDEPT',   u'ScreenBottom',    u'ScreenDepth',
            u'StateCode', u'StationComment',      u'StationId',
          u'StationName',    u'StationType',          u'UTM_X',
                u'UTM_Y',            u'WIN',             u'WL',
               u'WLDate',          u'WRNUM'],
      dtype='object')

In [123]:
GWStat.drop([u'Aquifer', u'AquiferType', u'CountyCode',u'ElevAcc', u'ElevAccUnit', u'ElevMeth',
             u'ElevRef', u'ElevUnit',u'Geology', u'HUC8', u'HoleDUnit', u'SCREENDEPT',
             u'HorAcc', u'HorAccUnit', u'HorCollMeth', u'HorRef'], inplace=True, axis=1)

C:\Python27\ArcGIS10.3\lib\site-packages\IPython\kernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [124]:
GWStat['WRNUM2'], GWStat['Dist'], GWStat['Diam'], GWStat['DepthWR'], GWStat['DrillDate'], GWStat['Loc'], GWStat['WIN'] = zip(*GWStat[['UTM_X','UTM_Y','ConstDate','Depth']].apply(lambda x: winmatch(x),1))

C:\Python27\ArcGIS10.3\lib\site-packages\IPython\kernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [125]:
pipergrps = piperdataND.groupby(['StationId'], squeeze=True).median()
pipergrps.drop(pipergrps.columns[[0]], axis=1, inplace=True)

In [126]:
GWStat.to_csv(rootname+'GWStations.csv',index_label='objid')

In [127]:
pipergrps.to_csv(rootname+'avgpiper.csv',index_label='objid')
piperdata.to_csv(rootname+'piper.csv',index_label='objid')

GWRes.to_csv(rootname+'GWResults.csv',index_label='objid')

In [128]:
pipergrps

ParAbb,2H,Ag_diss,Ag_tot,Al_tot,As_,As_diss,As_tot,B,Ba,Be,...,Geology,Lat_Y,Lon_X,SCREENDEPT,ScreenBottom,ScreenDepth,UTM_X,UTM_Y,WIN,WL
StationId,,,,,,,,,,,,,,,,,,,,,
414024111481101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,41.672975,-111.803943,NaN,NaN,NaN,NaN,NaN,NaN,NaN
414115111490301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0377,0.175445,NaN,...,NaN,41.687671,-111.818130,NaN,369,269,NaN,NaN,2722,180
414143111495501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,41.695106,-111.832618,NaN,230,195,NaN,NaN,437364,17
414213111493101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.130800,NaN,...,NaN,41.703693,-111.826497,NaN,215,180,NaN,NaN,2815,41
EPALeg-0300101,NaN,NaN,0.002,NaN,NaN,NaN,0.00100,0.0550,0.000050,NaN,...,NaN,41.642083,-111.819028,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EPALeg-0300102,NaN,NaN,0.002,NaN,NaN,NaN,0.00100,0.0500,0.000075,NaN,...,NaN,41.642778,-111.818333,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EPALeg-0300103,NaN,NaN,0.002,NaN,NaN,NaN,0.00050,0.0550,0.000090,NaN,...,NaN,41.641389,-111.817917,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EPALeg-0300104,NaN,NaN,0.002,NaN,NaN,0.0003,0.00050,0.0100,0.025025,NaN,...,NaN,41.658611,-111.831528,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EPALeg-0300201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0100,0.050000,NaN,...,NaN,41.854722,-111.862500,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [129]:
writer = pd.ExcelWriter(rootname + "combined_out.xlsx", engine="xlsxwriter")
Stat.to_excel(writer, "stations", index=False)
results.to_excel(writer, "results", index=False)
GWStat.to_excel(writer, 'GWStations',index=False)
GWRes.to_excel(writer, 'GWResults',index=False)
piperdata.to_excel(writer, "piperpivot")
writer.save()

In [130]:
sorted(list(results.Param.unique()))

['Alkalinity',
 'Alkalinity, Carbonate',
 'Alkalinity, Carbonate as CaCO3',
 'Alkalinity, Total',
 'Alpha, gross',
 'Aluminum',
 'Aluminum, Total',
 'Ammonia',
 'Ammonia as N',
 'Ammonium',
 'Antimony',
 'Antimony, Total',
 'Arsenic',
 'Arsenic, Dissolved',
 'Arsenic, Total',
 'As_',
 'Barium',
 'Barium, Dissolved',
 'Barium, Total',
 'Barometric pressure',
 'Berylium',
 'Beryllium',
 'Beryllium, Total',
 'Beta, gross',
 'Bicarbonate',
 'Bicarbonate as HCO3',
 'Biochemical oxygen demand, standard conditions',
 'Bod, 5 Day, 20 Deg C',
 'Boron',
 'Boron, Dissolved',
 'Boron, Total',
 'Bromide',
 'CO2',
 'CO3',
 'Ca',
 'Cadmium',
 'Cadmium, Dissolved',
 'Cadmium, Total',
 'Calcium',
 'Calcium carbonate',
 'Calcium, Dissolved',
 'Carbon',
 'Carbon Dioxide',
 'Carbon dioxide',
 'Carbon, Dissolved Inorganic',
 'Carbon, Dissolved Organic',
 'Carbon, Total',
 'Carbonaceous biochemical oxygen demand, standard conditions',
 'Carbonate',
 'Carbonate (CO3)',
 'Chemical oxygen demand',
 'Chloride',

In [169]:
GWSTID = list(GWStat[GWStat['StationName'].str.contains('Glenrid')].StationId.values)
GWSTID.append('USGS-414115111490301')
GWSTID

['SDWIS3117.0WS004', u'414115111490301', 'USGS-414115111490301']

In [170]:
Nitrate = GWRes[GWRes['ParAbb'].isin(['N','NO2','NO3','NH4'])]

In [171]:
Nitrate[Nitrate['StationId'].isin(GWSTID)]['ResultValue'].plot()

TypeError: Empty 'DataFrame': no numeric data to plot